# 第10回 Monte-Carlo Tree Search 3

## Monte-Carlo Tree Search を強化するには

前回まででMCTSの基本的な動作を紹介した。
しかしここまでに説明した方法では、3目並べや重力付き4目並べでさえそれほど強いプログラムであるとは言えない。
今回は、MCTSの性能を上げる方法を紹介する。

### Rollout の強化

今まで説明してきたMCTSでは、節点の評価を乱数を用いた Rollout (simulation, playout) で行っていた。
しかし、実際にこの方法の性能を見てみると、 Rollout の回数が少ないときには簡単なゲームでさえそれほど強くない。
Dijkstra法をヒューリスティック関数で強化するように、MCTSを強化する方法がいくつかある。

代表的な物は Rollout を強化することである。
完全にランダムな Rollout ではなく、何らかの方法で重要な手順をたどるように変更することによって
少ない rollout 回数でもより良い回を得ることができる。

#### 手作りの Rollout

まずはわかりやすい例を挙げよう。
3目並べの Rollout の動作を変更し、優先度順に以下のルールに従うとする。
1. 自分の3が作れるならそれを選ぶ
1. 相手の3を止められるなら止める手を選ぶ
1. それ以外の手をランダムに選ぶ
このようにするだけで Rollout の質がかなり向上する。

こうやって人間が考えた手作りの (hand-crafted) Rollout でMCTSを強化するのは
MCTSの初期から使われている。
実際に、人間の初段レベルに初めて到達した囲碁プログラムでは、
20個ほどのパターンから着手を選び、パターンにマッチする手がない場合に
残りの手を選ぶ、という Rollout が使われていた [Wang and Gelly 2007]。

> Y. Wang and S. Gelly, “Modifications of UCT and sequence-like simulations for Monte-Carlo Go,” in Proc. IEEE Symp. Comput. Intell. Games, 2007, pp. 175–182.

これは **苦手な形の木** への対応にもある程度、有効である。
MCTSは確率的な動作をするため、以下の Fig. 1 のような細い正解をたどるのが苦手であることは前回も紹介した。

<!-- | ![narrow_tree](narrow_tree.png)| -->
| <img src="narrow_tree.png" width="300"/> |
|:--:|
| <b>Fig. 1, 細い正解</b>|

このような細い正解は、例えばチェスや将棋には頻繁に出てくるが、囲碁ではそれほど頻度が高くない。
そのために Rollout を用いた MCTS は囲碁には適していたが将棋などには不向きだった。
しかし囲碁にも実は一本道が出現することがある。
「シチョウ知らずに囲碁打つな」と言われる有名なシチョウという形があり、これは通常は完全な一本道で人間なら簡単に認識できる。

| <img src="ladder.png" width="300"/> |
|:--:|
| <b>Fig. 2, 「シチョウ」の形</b>|

しかし確率的な Rollout を行っているとこのような一本道を最後までたどる確率が低いために正確な報酬が得られない。
そこで「シチョウ」の一本道を最後までたどるようなルールを Rollout に加えるとかなり強さが向上する。
このように、簡単にルール化できるような一本道であれば手作りの Rollout でかなり対応できる。
Rollout の工夫でMCTSの弱点を補っていると言える。


#### 機械学習による Rollout

別の方法として、 Rollout 中の選択肢を機械学習を用いて確率的に選択させるという方法もある。
例としては人間の対局を記録した棋譜から学習して、人間が選びそうな手を高い確率で選択させるという物である。

| <img src="go_features.png" width="600"/> |
|:--:|
| <b>Fig. 3, 囲碁の特徴と機械学習の例</b>|

簡単なイメージを Fig. 3 に示した。
人間が選びそうな候補手に高い確率を与える関数を機械学習で作成したいのだが、
ここでは4つの **特徴 (feature)** に **重み** をかけて算出する例を示している。
それぞれの手の特徴と重みから確率を計算することができる。
この場合は、何らかの機械学習手法によって人間の対局から重みの値を調整し、人間の着手を予測できるようにする。
学習した結果を Rollout に使うことによって、人間が選びそうな手を高い確率で選ぶことができる。








### 機械学習の利用

上記のように機械学習などで候補手に優先度をつけたり、1回も Rollout をしていない状態である程度の評価をすることができる。
これを利用したテクニックが考えられる。

#### Progressive Widening

今まで説明したことに従うと、UCB1のように通常は数式の分母に訪問回数があるので、訪問回数がゼロの場合は値が無限大となる。
そこで自然と、全ての節点で1回ずつ Rollout が行われることになる。
しかし、優先順位が低い手でも Rollout を行うことは無駄な可能性が高い。
候補手の優先順位を付けることができれば展開直後で全ての子節点の訪問回数が 0 の状態でも工夫ができる。

特に分岐数が大きい問題に対して、 **progressive widening** という方法が使われる。
これは訪問回数が少ない節点には、優先順位の上位$k$個しか子節点が存在しないと考える方法である。
$k$ は訪問回数 $s$ が増えるにつれて増える何らかの関数で定義される。

Progressive widening は分岐数が大きい問題に対しては標準的に使われるテクニックである。
優先順位の精度が高ければ大きな効果がある。


#### 有望な手を選ぶ数式の変更

$i$番目の候補手について、機械学習モデルが出力する確率を $p_i$ とする。
例えば囲碁プログラム AlphaGo は探索の際に UCB1の代わりに以下の様な式を用いる PUCT という方法を使った。

$ \frac{w_i}{s_i} + c_{puct} p_i \frac{\sqrt{t}}{1 + s_i} $

前回の定義と同様に、$i$番目の候補手の訪問回数が $s_i$ 回、報酬の合計が $w_i$、また $t$ は親節点の rollout 回数の合計とする。
定数 $c_{puct}$ は何らかの方法で調整する。
log がないことなどを含めて、かなりUCB1とは違う形をしている。


### 末端まで到達したらどうするか

3目並べや重力付き4目並べのような小さな問題だと、探索木が末端まで到達することがある。

| <img src="tictactoe2.png" width="400"/> |
|:--:|
| <b>Fig. 4, 3目並べなら簡単に末端に到達する</b>|

このときに、単純な方法としては末端での評価値を報酬として返すというものがある。
この方法でも最善解への収束は保証されているため、いずれは正しい答えが得られるはずなのだが、小さい問題でこの方法をとることは実際には無駄である。

| <img src="tictactoe3.png" width="800"/> |
|:--:|
| <b>Fig. 5, 末端の値を上へ返す</b>|

そこで、末端についた際にはその節点に末端であることを示すフラグを立てて、今後はRolloutに対象にはしないようにする、という方法が考えられる。
3目並べの場合、末端についた場合、評価値はもう変化しないので Rollout を行うのは無駄である。
（末端であっても報酬が確率的に得られる問題なら、この方法は使えないことに注意。）
Fig. 5 で赤い枠で囲われている局面は末端であり、評価値が確定している。
さらに、子節点全てが赤い枠で囲われた場合、親節点も評価値を確定できるので末端と見なしやはり赤枠で囲う。
これを繰り返すと、最終的には Minimax 探索と同様に全ての節点を探索することになる。

実際にはこのように小さな問題に MCTS を使うのは無駄なことが多い。
しかし、探索木のサイズは大きいが深さが不均衡で、浅い末端節点が混ざっているような場合にはこの工夫をしているかどうかで性能に大きな差が出る。


## 第10回課題

- 10-1. 【Rollout強化 3目並べ】 上で説明したように、3目並べの Rollout として、以下の動作をするものを実装せよ。
    「1, もし3を作れるならその手を選ぶ 2, もし相手の3を阻止できるならその手を選ぶ 3, それ以外の手をランダムに選ぶ。」
- 10-2. 【強化された Rollout で原始モンテカルロ探索】 第8回課題で作成した原始モンテカルロ3目並べの Rollout を 10-1 で作った物に変更し、強さを比較するなど動作を観察せよ。
- 10-3. 【強化された Rollout で MCTS 3目並べ】 第9回課題で作成した原始モンテカルロ3目並べの Rollout を 10-1 で作った物に変更し、強さを比較するなど動作を観察せよ。
- 10-4. (発展課題) 【強化された Rollout による重力付き4目並べ】 重力付き4目並べについて、10-1と同様に、「1, 自分の4を作る、2, 相手の4を止める、3, それ以外をランダム」という Rollout を行うプログラムを作成せよ。また、作成した Rollout を用いて MCTS で重力付き4目並べをプレイするプログラムを作成し、強さを比較するなど動作を観察せよ。

# 課題1

In [9]:
class TicTacToeNode:
    def rule_choice(self):
            if not self.state.is_terminal():
                blank=self.state.blank()
                for m in blank:
                    if self.state.make_next_state(m,player_id=self.player).is_win(player_id=self.player):
                        return m
                for m in blank:
                    if self.state.make_next_state(m,player_id=1-self.player).is_win(player_id=1-self.player):
                        return m
            return False

# 課題2

In [28]:
# tic-tac-toe playout
import copy
import random
import numpy as np


class mcts_node:
    def __init__(self,parent=None,move=None,state=None,player_id=None,game=None):
        self.childnode=[]
        self.move=move
        self.state=state
        self.visit=0
        self.value=0
        self.parent=parent
        self.player_id=player_id
        self.game=game
        if game.is_terminal(state):
            self.untried_node=[]
        else:
            self.untried_node=game.board_blank(state)
        
        
    def select_by_ucb(self):
        reward_li=[]
        for i in self.childnode:
            ucb=(i.value/i.visit)+np.sqrt(2*np.log(self.visit)/i.visit)
            reward_li.append(ucb)
        if self.player_id ==0:
            num=np.argmax(reward_li)
        else:
            num=np.argmin(reward_li)
        return self.childnode[num]
        
    
    def expand(self,move,state):
        child=mcts_node(parent=self,move=move,state=state,player_id=1-self.player_id,game=self.game)
        self.untried_node.remove(move)
        self.childnode.append(child)
        return child
        
        
    def backpropagation(self,reward):
        self.visit+=1
        self.value+=reward
    
        
        
class TicTacToe:
    def __init__(self):
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in state:
            for j in i:
                if j==' ':
                    return False
        return True

    def is_win(self, state, player_id):
        a=self.players[player_id]
        li=[]
        for i in range(3):
            if state[i]==[a,a,a]:
                return True
            for j in range(3):
                if state[i][j]==a:
                    li.append([i,j])
        pat=[[[i,j] for i in [0,1,2]] for j in [0,1,2]] +[[[0,0],[1,1],[2,2]]]+[[[0,2],[1,1],[2,0]]]
        for i in pat:
            x=0
            for j in i:
                if j in li:
                    x+=1
            if x==3:
                return True
        return False
    
    def reward(self,state):
        if self.is_draw(state):
            return 0
        elif self.is_win(state,0):
            return 1
        elif self.is_win(state,1):
            return -1
        
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        x, y = move
        char = self.players[player_id]
        new_state[x][y] = char
        return new_state
    
    def board_blank(self,state):
        blank=[]
        for i in range(3):
            for j in range(3):
                if state[i][j]==' ':
                    blank.append([i,j])
        return blank
    
    def rule_choice(self,state,player_id):
            if not self.is_terminal(state):
                blank=self.board_blank(state)
                for m in blank:
                    if self.is_win(state=self.make_move(state, m, player_id=player_id),player_id=player_id):
                        return m
                for m in blank:
                    if self.is_win(state=self.make_move(state, m, player_id=1-player_id),player_id=1-player_id):
                        return m
            return False


def mcts(game,begin_state,iteration,use_rule=False):
    mstnode=mcts_node(parent=None,move=None,state=begin_state,player_id=0,game=game)
    
    for i in range(iteration):
        node=mstnode
        state=begin_state
        id=mstnode.player_id

                
        while node.untried_node == [] and node.childnode !=[]:
            node=node.select_by_ucb()
            state=game.make_move(state,node.move,id)
            id=1-id
            

        if node.untried_node != []:
            m=random.choice(node.untried_node)
            state=game.make_move(state,m,id)
            id=1-id
            node=node.expand(m,state)

        if use_rule==False:
            while not game.is_terminal(state) and game.board_blank(state) != []:
                state=game.make_move(state,move=random.choice(game.board_blank(state)),player_id=id)
                id=1-id
        else:
            while not game.is_terminal(state) and game.board_blank(state) != []:
                if game.rule_choice(state,id):
                    m=game.rule_choice(state,id)
                else:
                    m=random.choice(game.board_blank(state))
                state=game.make_move(state,move=m,player_id=id)
                id=1-id

        while node is not None:
            node.backpropagation(game.reward(state))
            node=node.parent
        
    return mstnode

print('第8回課題で作成した原始モンテカルロ3目並べ:')
game = TicTacToe()
root_state = [[' ',' ','X'],
              [' ','O',' '],
              ['X',' ','O']]
mst=mcts(game,root_state,100,use_rule=False)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.state,':',i.value/i.visit)
    

print('第10回課題で作成したRollout強化3目並べ:')
mst=mcts(game,root_state,100,use_rule=True)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.state,':',i.value/i.visit)


第8回課題で作成した原始モンテカルロ3目並べ:
mst.value: 0.63
[[' ', ' ', 'X'], [' ', 'O', 'X'], ['X', ' ', 'O']] : -0.14285714285714285
[[' ', ' ', 'X'], [' ', 'O', ' '], ['X', 'X', 'O']] : -0.5
[[' ', 'X', 'X'], [' ', 'O', ' '], ['X', ' ', 'O']] : -1.0
[[' ', ' ', 'X'], ['X', 'O', ' '], ['X', ' ', 'O']] : -0.5
[['X', ' ', 'X'], [' ', 'O', ' '], ['X', ' ', 'O']] : 0.8433734939759037
第10回課題で作成したRollout強化3目並べ:
mst.value: 0.73
[[' ', 'X', 'X'], [' ', 'O', ' '], ['X', ' ', 'O']] : -1.0
[['X', ' ', 'X'], [' ', 'O', ' '], ['X', ' ', 'O']] : 0.8804347826086957
[[' ', ' ', 'X'], ['X', 'O', ' '], ['X', ' ', 'O']] : -1.0
[[' ', ' ', 'X'], [' ', 'O', 'X'], ['X', ' ', 'O']] : -1.0
[[' ', ' ', 'X'], [' ', 'O', ' '], ['X', 'X', 'O']] : -1.0


# 課題3

In [29]:
players_str = ['X', 'O']

class TicTacToeState:
    def __init__(self, board, player):
        '''
        initial board
        [[' ',' ',' '],
         [' ',' ',' '],
         [' ',' ',' ']]                           
        '''
        self.board = board
        self.player = player

    def show_board(self):
        for i in range(3):
            for j in range(3):
                print(self.board[i][j], end="")
            print('|')
        
    def make_next_state(self, move, player_id=None):
        '''
        return next state
        '''
        if player_id == None:
            player_id = self.player
        new_board = copy.deepcopy(self.board)
        x, y = move
        char = players_str[player_id]
        new_board[x][y] = char
        return TicTacToeState(new_board, 1-player_id)
    
    def is_terminal(self):
        '''
        return true if state is terminal
        '''
        if self.is_win(self.player):
            return True
        elif self.is_win(1-self.player):
            return True
        elif self.is_draw():
            return True
        return False
    
    def score(self, player_id = None):
        if player_id == None:
            player_id = self.player
        if self.is_win(player_id):
            return 1
        elif self.is_win(1 - player_id):
            return -1
        elif self.is_draw():
            return 0
        assert(False)
        
    def is_draw(self):
        for i in range(3):
            for j in range(3):
                if self.board[i][j]==' ':
                    return False
        return True

    def is_win(self, player_id = None):
        if player_id == None:
            player_id = self.player
        char = players_str[player_id]
        for i in range(3):
            win_flag = True
            for j in range(3):
                if self.board[i][j]!=char:
                    win_flag = False
                    break
            if win_flag:
                return True
            
        for i in range(3):
            win_flag = True
            for j in range(3):
                if self.board[j][i]!=char:
                    win_flag = False
                    break
            if win_flag:
                return True

        win_flag = True
        for i in range(3):
            if self.board[i][i]!=char:
                win_flag = False
                break
        if win_flag:
            return True

        win_flag = True
        for i in range(3):
            if self.board[2-i][i]!=char:
                win_flag = False
                break
        if win_flag:
            return True

        return False
    
    def blank(self):
        blank=[]
        for i in range(3):
            for j in range(3):
                if self.board[i][j]==' ':
                    blank.append([i,j])
        return blank

class TicTacToeNode:
    def __init__(self, state, node_id, player, w, s):
        '''
        state = board state
        player: player id 0 ("X") or 1 ("O")
        w: total reward
        s: number of visits
        '''
        self.state = state
        self.node_id = node_id
        self.player=player
        self.w = w # total reward
        self.s = s # number of visits
        self.expanded = False
        self.nu_child = 0
        self.children = []

    def is_root_node(self):
        if self.node_id == (0,):
            return True
        else:
            return False

    def print(self):
        print(self.node_id, self.w, self.s)
        #self.state.show_board()
        #for c in self.children:
        #    print(c)
        
        
    def rule_choice(self):
        if not self.state.is_terminal():
            blank=self.state.blank()
            for m in blank:
                if self.state.make_next_state(m,player_id=self.player).is_win(player_id=self.player):
                    return m
            for m in blank:
                if self.state.make_next_state(m,player_id=1-self.player).is_win(player_id=1-self.player):
                    return m
        return False
        

In [32]:
import copy
import random
import numpy as np

class MCTS:
    def __init__(self, expand=1,exploration=1,nu_rollout=10):
        self.tree = {}
        # 以下の定数は適当に変更して動作を観察すると良い
        self.expand_threshold = expand
        self.exploration_constant = exploration
        self.nu_rollout = nu_rollout

    def selection(self, root_node):
        node = root_node
        node_id = root_node.node_id
        path = [node_id]
        '''
        例えば、ここを実装せよ
        '''
        player=root_node.player
        while node.nu_child>0:
            value=[]
            for i in node.children:
                j=self.tree[i]
                value.append(self.UCB1(j.w,j.s,node.s))
            if root_node.player==player:    
                num=np.argmax(value)
            else:
                num=np.argmin(value)
            path.append(node.children[num])
            node=self.tree[node.children[num]]
            player=1-player
        return path

    def UCB1(self, w, s, t):
        if s == 0:
            return 10000
        mean_reward = w/s
        bias_value = np.sqrt(np.log(t)/s)
        ucb1_value = mean_reward + self.exploration_constant * bias_value
        return ucb1_value
        
    def expansion(self, node):
        '''
        generate all children (legal moves)
        '''
        next_player = 1 - node.state.player

        if node.s < self.expand_threshold and not node.is_root_node():
            return node
        
        num = 0
        for i in range(3):
            for j in range(3):
                if node.state.board[i][j]==' ':
                    move = (i, j)
                    action_id = i*3 + j
                    num += 1
                    child_id = node.node_id + (action_id,)
                    node.children.append(child_id)
                    child_state = node.state.make_next_state(move)
                    child_node = TicTacToeNode(child_state, child_id, next_player, 0, 0)
                    self.tree[child_id] = child_node
                    
        node.expanded = True
        node.nu_child = num
        if len(node.children) == 0:
            return node
        leaf_id = random.choice(node.children)
        return self.tree[leaf_id]
        

    def simulation(self, node,root_player,use_rule):
        state = node.state
        '''
        前回の課題 8-3 を参考にして良いが、データ構造が変わっていることに注意
        '''
        while not state.is_terminal():
            blank=state.blank()
            if use_rule and node.rule_choice():
                m = node.rule_choice()
            else:
                m=random.choice(blank)
            state=state.make_next_state(m)
        return state.score(root_player)
            

    def backpropagation(self, path, reward):
        '''
        以下を実装せよ
        '''
        for i in range(len(path)-1,-1,-1):
            node=self.tree[path[i]]
            node.s+=1
            node.w+=reward
            

    def start_search(self, root_board, player,use_rule):
        root_node_id = (0, ) # 要素が1個のtupleを作っている
        root_state = TicTacToeState(root_board, player)
        root_node = TicTacToeNode(root_state, root_node_id, player, 0, 0)
        self.tree[root_node_id] = root_node
        
        if root_node.state.is_terminal():
            score = root_node.state.score()
            return None, score

        for n in range(self.nu_rollout):
            path = self.selection(root_node)
            leaf_node_id = path[-1]
            leaf_node = self.tree[leaf_node_id]
            leaf_node = self.expansion(leaf_node)
            reward = self.simulation(leaf_node,player,use_rule)
            self.backpropagation(path, reward)

        best_value = -10000
        best_child_id = None
        assert(root_node.nu_child >= 1)
        for child_id in root_node.children:
            child_node = self.tree[child_id]
            value = child_node.w
            print(child_id, (child_node.w/child_node.s))
            if value > best_value:
                best_value = value
                best_child_id = child_id

        return best_child_id, (root_node.w/root_node.s)

root_board = [[' ',' ','X'],
              [' ','O',' '],
              ['X',' ','O']]                           


print('第9回課題で作成した原始モンテカルロ3目並べの Rollout:')
search = MCTS(nu_rollout=100)
search.start_search(root_board, 0,use_rule=False)

print('第10回課題で作成した強化された Rollout で MCTS 3目並べ の Rollout:')
search = MCTS(nu_rollout=100)
search.start_search(root_board, 0,use_rule=True)

第9回課題で作成した原始モンテカルロ3目並べの Rollout:
(0, 0) 0.9120879120879121
(0, 1) -1.0
(0, 3) -0.3333333333333333
(0, 5) -1.0
(0, 7) -1.0
第10回課題で作成した強化された Rollout で MCTS 3目並べ の Rollout:
(0, 0) 0.9368421052631579
(0, 1) -1.0
(0, 3) -1.0
(0, 5) -1.0
(0, 7) -1.0


((0, 0), 0.84)

In [35]:
root_board = [[' ',' ',' '],
              [' ',' ',' '],
              [' ',' ',' ']]                           

search = MCTS(nu_rollout=100)
search.start_search(root_board, 0,use_rule=True)

(0, 0) -0.5
(0, 1) -1.0
(0, 2) 0.7419354838709677
(0, 3) -0.3333333333333333
(0, 4) 0.6666666666666666
(0, 5) 0.5625
(0, 6) 0.2857142857142857
(0, 7) 0.3333333333333333
(0, 8) -1.0


((0, 2), 0.5)

# 課題4

In [38]:
import copy
import random
import numpy as np

class ConnectFour:
    def __init__(self):
        # 7x6
        self.players = ['X', 'O']
        
    def is_terminal(self, state):
        if self.is_draw(state):
            return True
        elif self.is_win(state,0) or self.is_win(state,1):
            return True
        return False
        
    def is_draw(self, state):
        if self.is_win(state,0) or self.is_win(state,1):
            return False
        for i in range(7):
            if state[i][0]==' ':
                return False
        return True

    def is_win(self, state, player_id):
        li=[]
        for i in range(7):
            for j in range(6):
                if state[i][j]==self.players[player_id]:
                    li.append([i,j])
        for i in li:
            if [i[0]+1,i[1]] in li and [i[0]+2,i[1]] in li and [i[0]+3,i[1]] in li:
                return True
            if [i[0],i[1]+1] in li and [i[0],i[1]+2] in li and [i[0],i[1]+3] in li:
                return True
            if [i[0]+1,i[1]+1] in li and [i[0]+2,i[1]+2] in li and [i[0]+3,i[1]+3] in li:
                return True
            if [i[0]-1,i[1]+1] in li and [i[0]-2,i[1]+2] in li and [i[0]-3,i[1]+3] in li:
                return True
        return False
    
    def reward(self,state):
        if self.is_draw(state):
            return 0
        elif self.is_win(state,0):
            return 1
        elif self.is_win(state,1):
            return -1
        
    def make_move(self, state, move, player_id):
        new_state = copy.deepcopy(state)
        for i in range(6):
            if state[move][i] != ' ':
                char = self.players[player_id]
                new_state[move][i-1] = char
                return new_state
        char = self.players[player_id]
        new_state[move][5] = char
        return new_state

            
    def board_blank(self,state):
        blank=[]
        for i in range(7):
            if state[i][0]==' ':
                blank.append(i)
        return blank
    
    def show_board(self,state):
        for i in state:
            print(i)
    
    def rule_choice(self,state,player_id):
            if not self.is_terminal(state):
                blank=self.board_blank(state)
                for m in blank:
                    if self.is_win(state=self.make_move(state, m, player_id=player_id),player_id=player_id):
                        return m
                for m in blank:
                    if self.is_win(state=self.make_move(state, m, player_id=1-player_id),player_id=1-player_id):
                        return m
            return False

class mcts_node:
    def __init__(self,parent=None,move=None,state=None,player_id=None,game=None):
        self.childnode=[]
        self.move=move
        self.state=state
        self.visit=0
        self.value=0
        self.parent=parent
        self.player_id=player_id
        self.game=game
        if game.is_terminal(state):
            self.untried_node=[]
        else:
            self.untried_node=game.board_blank(state)
        
        
    def select_by_ucb(self):
        reward_li=[]
        for i in self.childnode:
            ucb=(i.value/i.visit)+np.sqrt(2*np.log(self.visit)/i.visit)
            reward_li.append(ucb)
        if self.player_id ==0:
            num=np.argmax(reward_li)
        else:
            num=np.argmin(reward_li)
        return self.childnode[num]
        
    
    def expand(self,move,state):
        child=mcts_node(parent=self,move=move,state=state,player_id=1-self.player_id,game=self.game)
        self.untried_node.remove(move)
        self.childnode.append(child)
        return child
        
        
    def backpropagation(self,reward):
        self.visit+=1
        self.value+=reward
        


def mcts(game,begin_state,iteration,use_rule=False):
    mstnode=mcts_node(parent=None,move=None,state=begin_state,player_id=0,game=game)
    
    for i in range(iteration):
        node=mstnode
        state=begin_state
        id=mstnode.player_id

                
        while node.untried_node == [] and node.childnode !=[]:
            node=node.select_by_ucb()
            state=game.make_move(state,node.move,id)
            id=1-id
            

        if node.untried_node != []:
            m=random.choice(node.untried_node)
            state=game.make_move(state,m,id)
            id=1-id
            node=node.expand(m,state)

        if use_rule==False:
            while not game.is_terminal(state) and game.board_blank(state) != []:
                state=game.make_move(state,move=random.choice(game.board_blank(state)),player_id=id)
                id=1-id
        else:
            while not game.is_terminal(state) and game.board_blank(state) != []:
                if game.rule_choice(state,id):
                    m=game.rule_choice(state,id)
                else:
                    m=random.choice(game.board_blank(state))
                state=game.make_move(state,move=m,player_id=id)
                id=1-id

        while node is not None:
            node.backpropagation(game.reward(state))
            node=node.parent
        
        li=[]
        for i in mstnode.childnode:
            li.append(i.value/i.visit)
    return mstnode, mstnode.childnode[np.argmax(li)]
    

In [40]:
game = ConnectFour()
root_state = [[' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' '],
              [' ',' ',' ',' ',' ',' ']]  

print('第9回課題で作成した Rollout を用いて MCTS で重力付き4目並べ:')
mst,best_child=mcts(game=game,begin_state=root_state,iteration=200,use_rule=False)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.move,':',i.value/i.visit)
print('--------------------------------')
game.show_board(best_child.state)

print('第10回課題で強化された Rollout を用いて MCTS で重力付き4目並べ:')
mst,best_child=mcts(game=game,begin_state=root_state,iteration=200,use_rule=True)
print('mst.value:',mst.value/mst.visit)
for i in mst.childnode:
    print(i.move,':',i.value/i.visit)
print('--------------------------------')
game.show_board(best_child.state)

第9回課題で作成した Rollout を用いて MCTS で重力付き4目並べ:
mst.value: 0.24
5 : 0.3125
4 : 0.2571428571428571
1 : -0.16666666666666666
0 : 0.21428571428571427
2 : 0.22580645161290322
6 : -0.42857142857142855
3 : 0.38181818181818183
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
第10回課題で強化された Rollout を用いて MCTS で重力付き4目並べ:
mst.value: 0.02
3 : -0.08695652173913043
1 : -0.3333333333333333
0 : -0.09090909090909091
4 : 0.0
6 : -0.043478260869565216
2 : 0.0
5 : 0.20967741935483872
--------------------------------
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', ' ']
[' ', ' ', ' ', ' ', ' ', 'X']
[' ', ' ', ' ', ' ', ' ', ' ']


参考文献
1. "Artificial Intelligence: A Modern Approach, 4th Global ed.", by Stuart Russell and Peter Norvig  
   http://aima.cs.berkeley.edu/index.html
1. "ヒューリスティック探索入門", 陣内 佑  
   https://jinnaiyuu.github.io/pdf/textbook.pdf
1. C. B. Browne et al., “A Survey of Monte Carlo Tree Search Methods,” IEEE Transactions on Computational Intelligence and AI in Games, vol. 4, no. 1, pp. 1–43, doi: 10.1109/TCIAIG.2012.2186810.